In [118]:
import pandas as pd
import os
import numpy as np
from ensure import ensure_annotations
import re
import warnings
warnings.filterwarnings('ignore')

In [119]:
print(os.getcwd())
print(os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem'))
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem
None
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [120]:
df = pd.read_csv(r'data\raw_data\one_plus_mobile_new_data.csv',index_col=None)

In [121]:
df.drop_duplicates(inplace=True,ignore_index=True)

## Price

In [122]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['price']):
    i = str(i).replace('₹','')
    i = str(i).replace(',','')
    df['price'][Index] = i

In [123]:
df['price'] = df['price'].astype('Int16')

In [124]:
df['price'] = df['price'].abs() # this is to remove the negative values from price feature

## Ratings

In [125]:
df['ratings'].dtype

dtype('float64')

## Ram

In [126]:
def extract_ram(row):
    ram = row.split()[2]
    return ram

df['ram'] = df['storage_ram'].apply(func=extract_ram)

In [127]:
import warnings
warnings.filterwarnings('ignore')
count = 0
for i in range(0,len(df['ram'])):
    # df['ram'][i]= df['ram'].iloc[i].replace('MBRAM','')
    df['ram'][i]= df['ram'].iloc[i].replace('GBRAM','')
    count+=1
print(f'looped till {count} index')

looped till 188 index


In [128]:
df['ram'] = df['ram'].astype('Int16')

## storage

In [129]:
def extract_storage(row):
    storage = row.split()[1]
    return storage


df['storage'] = df['storage_ram'].apply(func=extract_storage)

In [130]:
import warnings
warnings.filterwarnings('ignore')
count = 0
for i in range(0,len(df['storage'])):
    df['storage'][i]= df['storage'].iloc[i].replace('Storage','')
    count+=1
print(f'looped till {count} index')

looped till 188 index


In [131]:
df['storage'] = df['storage'].astype('Int16')
## Removing storage_ram
df = df.drop(columns=['storage_ram','index'])

## Android Version

In [132]:
def extract_processor(text):
    pattern = r'.*Processor'
    match = re.findall(pattern, text)
    if match:
        return match[0]
    else:
        return text


In [133]:
df['os_processor'] = df['os_processor'].apply(extract_processor)

In [134]:
def android_version(text_):
    split = text_.split()
    if 'OxygenProcessor' in split:
        return 'OxygenProcessor'
    elif 'ANdroid' in split:
        return split[3]
    elif 'OxygenOSProcessor' in split:
        return 'OxygenOSProcessor'
    else:
        pattern=r'^\d+[Processor]|\d+[GProcessor]|\d++[.Processor]'
        found = re.findall(pattern,text_)
        if found:
            return found[0]
        else:
            return 'other androidVerion'

In [135]:
df['android_version'] = df['os_processor'].apply(android_version)

In [136]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['android_version']):
    # print(i)
    i = str(i).replace('P','')
    i = str(i).replace('.','')
    df['android_version'][Index] = i

## Processor

In [137]:
def processor(text):
    pattern=r'(?<=Type).*'
    search = re.search(pattern,text)
    if search != None:
        return ' '.join(re.findall(pattern,text))
    else:
        return 'not montioned'

In [138]:
df['processor'] = df['os_processor'].apply(processor)
df = df.drop('os_processor',axis=1)
df = df.drop('index',axis=1)


In [139]:
def processor_extract(text):
    splitted_sentence = text.split()
    if 'Mediatek' and 'Helio' in splitted_sentence:
        pattern = r'\d+[Processor]|\d+'
        string = ' '.join(splitted_sentence)
        match = re.search(pattern,string)
        if match == None:
            return 'Heilo'+str(splitted_sentence[2])
        else:
            find = re.findall(pattern,string)
            return 'Heilo'+''.join(find)


    elif 'Mediatek' and 'Dimensity' in splitted_sentence:
        pattern = r'(\d+)+[Processor]'
        string = ' '.join(splitted_sentence)
        match = re.search(pattern,string)
        if match == None:
            if 'Dimensity' == str(splitted_sentence[1]):
                return 'Dimensity'+str(splitted_sentence[2])
            else:
                return 'Dimensity'+str(splitted_sentence[1])
        
        else:
            find = re.findall(pattern,string)
            return 'Dimensity'+ ''.join(find)

    elif 'Snapdragon' and 'Gen' and '4' in splitted_sentence:
        return 'Gen4'   

    elif 'Snapdragon' and 'Gen' and '8' in splitted_sentence:         
        return 'Gen8'
    
    elif 'Unisoc' in splitted_sentence:
        
        if 'Unisoc' and 'Tiger' in splitted_sentence:
            pattern = r'[T]\d[Processor]'
            string = ' '.join(splitted_sentence)
            match = re.findall(pattern,string)
            return 'UnisocTiger' + ''.join(splitted_sentence[2])
        
        elif 'Unisoc' == splitted_sentence[0]:
            pattern = r'[T]\d.'
            string = ' '.join(splitted_sentence)
            match = re.findall(pattern,string)
            return 'Unisoc' + ''.join(splitted_sentence[1])

        
        pattern = r'T(\d+)Processor|\d+'
        string = ' '.join(splitted_sentence)
        match = re.findall(pattern,string)
        return 'Unisoc'+' '.join(match)
    

    elif 'Snapdragon' in splitted_sentence:
        pattern = r'\d+[Processor]'
        if 'Gen' and '4' and '8' not in splitted_sentence:
            string = ' '.join(splitted_sentence)
            match = re.search(pattern,string)
            if match == None:
                if 'Snapdragon' == str(splitted_sentence[1]):
                    return 'Snapdragon'+str(splitted_sentence[2])
                else:
                    return 'Snapdragon'+str(splitted_sentence[1])
            else:
                find = re.findall(pattern,string)
                return 'Snapdragon'+''.join(find)
    
    elif 'MediaTek' == splitted_sentence[0]:
        pattern = r'[P]\d.'
        string = ' '.join(splitted_sentence)
        match = re.findall(pattern,string)
        return 'MediaTek' + ' '.join(match)

    else:
        return 'not mentioned'      


In [140]:
df['processor']=df['processor'].apply(processor_extract)

## camera

In [141]:
def camera_extract(row):
    row_splitted = row.split()
    pattern = r'\d+MP'
    words = ' '.join(row_splitted)
    find_cam = re.findall(pattern,words)
    return ' '.join(find_cam)


In [142]:
df['camera'] = df['camera'].apply(camera_extract)

In [143]:
def camera(txt):
    lst = []
    splitted = txt.split()
    for i in splitted:
        make_int = i.replace('MP','')
        lst.append(int(make_int))
    return max(lst)


In [144]:
df['camera'] = df['camera'].apply(func=camera)
df['camera'] = df['camera'].astype('Int16')

## display

In [145]:
def display_extract(row):
    split_row = row.split()
    if 'LCD' in split_row:
        if 'IPS' in split_row and 'LCD' in split_row:
            return 'IPS LCD'
        return 'LCD'
    
    if 'OLED' in split_row:
        return 'OLED'
    
    if 'TFT' in split_row:
        return 'TFT'
    
    if 'LED' in split_row:
        return 'LED'

    # if 'Adreno' in split_row:
    #     return 'Adreno'
    if 'AMOLED' in split_row:
        if 'HD+' in split_row and 'AMOLED' in split_row:
            return 'HD+ AMOLED'
        return 'AMOLED'
    
    return 'other'


In [146]:
df['display'] = df['display'].apply(display_extract)

## network

In [147]:
df = df.drop(columns=['network'])

## battery

In [148]:
def extract_battery(row):
    return row.split()[1]

df['battery'] = df['battery'].apply(extract_battery)

In [149]:
df.reset_index(inplace=True)
count = 0
for i in range(0,len(df['battery'])):
    # print(i)
    df['battery'][i]= df['battery'].iloc[i].replace('Capacity','')
    # print(df['battery'][i])
    count+=1
print(f'looped till {count} index')

looped till 188 index


In [150]:
df['battery'] = df['battery'].astype('Int16')

In [152]:
df = df.drop(labels='index',axis=1)

In [154]:
df

,name,ratings,price,imgURL,camera,display,battery,ram,storage,android_version,processor
0,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",4.4,17676,https://rukminim2.flixcart.com/image/312/312/x...,64,other,5000,6,128,10,not mentioned
1,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",4.4,17691,https://rukminim2.flixcart.com/image/312/312/x...,64,other,5000,6,128,13,not mentioned
2,"OnePlus 11R 5G (Galactic Silver, 256 GB)",4.5,20846,https://rukminim2.flixcart.com/image/312/312/x...,50,other,5000,16,256,13,not mentioned
3,"OnePlus 11R 5G (Galactic Silver, 128 GB)",4.5,25802,https://rukminim2.flixcart.com/image/312/312/x...,50,other,5000,8,128,13,not mentioned
4,"OnePlus 11R 5G (Sonic Black, 256 GB)",4.5,21252,https://rukminim2.flixcart.com/image/312/312/x...,50,other,5000,16,256,13,not mentioned
...,...,...,...,...,...,...,...,...,...,...,...
183,"OnePlus N20 SE (BLUE OASIS, 64 GB)",4.2,15674,https://rukminim2.flixcart.com/image/312/312/x...,50,other,5000,4,64,12,not mentioned
184,"OnePlus 9 Pro 5G (Morning Mist, 128 GB)",3.8,17537,https://rukminim2.flixcart.com/image/312/312/x...,48,other,4500,8,128,11,not mentioned
185,"OnePlus 6 (Mirror Black, 128 GB)",4.2,17990,https://rukminim2.flixcart.com/image/312/312/x...,20,other,3300,8,128,8,not mentioned
186,"OnePlus 6T McLaren Edition (Speed Orange, 256 GB)",4.0,26990,https://rukminim2.flixcart.com/image/312/312/x...,20,other,3300,10,256,9,not mentioned


In [155]:
df.to_csv(r'data\cleaned_dataset\one_plus_cleaned_data.csv')